In [49]:
from selenium import webdriver
from lxml import etree
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
import pymysql
import re
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep

In [50]:
def CreateTable():
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='stocks',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    
    creatSql = """create table if not exists total_stocks(
                `id` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
                `code` INT NOT NULL UNIQUE,
                `name` VARCHAR(20) NOT NULL,
                `price` FLOAT NOT NULL,
                `up_down_rate` FLOAT NOT NULL,
                `up_down` FLOAT NOT NULL,
                `up_rate` FLOAT NOT NULL,
                `change_rate` FLOAT NOT NULL,
                `quantity_rate` FLOAT NOT NULL,
                `amplitude_rate` FLOAT NOT NULL,
                `transaction` VARCHAR(20) NOT NULL,
                `circulating` VARCHAR(20) NOT NULL,
                `circulation_market` VARCHAR(20) NOT NULL,
                `pe_ratio` VARCHAR(20) NOT NULL
                );"""
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(creatSql)
        connection.commit()
    finally:
        connection.close()


In [51]:
def writeData(trlist):
    connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db='stocks',
                             charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    sql = """
            INSERT total_stocks (`code`, `name`, `price`, `up_down_rate`, `up_down`, `up_rate`,
            `change_rate`, `quantity_rate`, `amplitude_rate`, `transaction`, `circulating`,
            `circulation_market`, `pe_ratio`)
            VALUES
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
           """
    try:
        with connection.cursor() as cursor:
            # execute执行一条，executemany批量插入
            cursor.executemany(sql, trlist)
        print("some data already insert database")
        connection.commit()
    finally:
        connection.close()

In [52]:
def is_number(string):
    pattern = re.compile(r'^[-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?$')
    return bool(pattern.match(string))

In [53]:
def main():
    CreateTable()
    
    # 添加启动参数，ban掉windows.navigator.webdriver
    option = webdriver.EdgeOptions()
    # 一种反爬机制，若为真，则无法正常显示内容
    option.add_argument('--disable-blink-features=AutomationControlled')
    driver = webdriver.Edge(options=option,keep_alive=True)
    driver.implicitly_wait(1)
    for page in range(1, 264):
        trlist = []
        driver.get(f'https://q.10jqka.com.cn/index/index/board/all/field/zdf/order/desc/page/{page}/ajax/1/')
        print(f"this is the {page}st page down")
        table = driver.find_element(By.XPATH, '//table[@class="m-table m-pager-table"]')
        tbody = table.find_element(By.XPATH, './tbody')
        trs = tbody.find_elements(By.XPATH, './tr')
        start = time.time()
        for tr in trs:
            tds = tr.find_elements(By.XPATH, './td')
            tdlist = []
            for i, td in enumerate(tds):
                if is_number(td.text) and i != 13:
                    if '.' in td.text:
                        text = float(td.text)
                    else:
                        text = int(td.text)
                    tdlist.append(text)
                elif '--' in td.text:
                    tdlist.append(None)
                else:
                    tdlist.append(td.text)
            trlist.append(tdlist[1: -1])
        end = time.time()
        print("the data of page append list used time: %.2f" % (end -  start))
        print(trlist)
        writeData(trlist)
    driver.close()

In [54]:
if __name__ == "__main__":
    main()

this is the 1st page down
the data of page append list used time: 6.71
[[301191, '菲菱科思', 110.14, 20.0, 18.36, 0.0, 39.87, 1.51, 21.74, '8.77亿', '2179.66万', '24.01亿', '42.19'], [300394, '天孚通信', 88.38, 20.0, 14.73, 0.0, 9.59, 2.22, 19.39, '29.00亿', '3.62亿', '319.51亿', '73.94'], [688498, '源杰科技', 183.92, 20.0, 30.65, 0.0, 23.64, 2.28, 21.48, '7.61亿', '1853.06万', '34.08亿', '402.34'], [300570, '太辰光', 39.2, 19.99, 6.53, 0.0, 16.16, 2.03, 20.48, '11.44亿', '1.94亿', '75.99亿', '62.90'], [300418, '昆仑万维', 36.56, 19.99, 6.09, 0.0, 16.07, 4.57, 18.84, '61.29亿', '11.06亿', '404.49亿', '62.01'], [688662, '富信科技', 38.55, 17.35, 5.7, 0.1, 14.68, 2.82, 20.97, '2.90亿', '5423.00万', '20.91亿', '亏损'], [301052, '果麦文化', 44.9, 16.62, 6.4, 0.54, 17.19, 5.68, 18.05, '4.05亿', '5470.22万', '24.56亿', '55.59'], [688702, '盛科通信-U', 59.95, 15.33, 7.97, 0.27, 33.55, 1.22, 18.37, '7.13亿', '3719.68万', '22.30亿', '346.60'], [300461, '田中精机', 23.69, 15.11, 3.11, -0.13, 11.93, 2.95, 16.57, '2.18亿', '8171.10万', '19.36亿', '453.46'], [6